**EXPLORATION ET NETTOYAGE DES DONNEES DE 4 TABLES  + MACHINE LEARNING** :

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)

**TABLE NAME BASIC :**

In [17]:
df_name_basics = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\name.basics.tsv', sep ='\t')

In [18]:
# REMPLACEMENT DES VALEURS VIDE PAR NA. 
# SUPPRESSION DES VALEURS NULL POUR CERTAINE COLONNE. 
# SUPPRESSION DES LIGNES DOUBLONS. 
# CHANGEMENT DE TYPE DES COLONNES.

df_name_basics = df_name_basics.replace(r"\N", pd.NA)               

df_name_basics = df_name_basics.dropna(subset = ['birthYear'])            
df_name_basics = df_name_basics.dropna(subset = ['primaryProfession'])    
df_name_basics = df_name_basics.dropna(subset = ['knownForTitles'])     

df_name_basics.drop_duplicates(inplace = True)                   

df_name_basics['nconst'] = df_name_basics['nconst'].astype("string")                         #changement type  ==> string
df_name_basics['primaryName'] = df_name_basics['primaryName'].astype("string")               #changement type  ==> string
df_name_basics['birthYear'] = df_name_basics['birthYear'].astype("int")                      #changement type  ==> int
df_name_basics['deathYear'] = df_name_basics['deathYear'].astype("string")                      #changement type  ==> int
df_name_basics['primaryProfession'] = df_name_basics['primaryProfession'].astype("string")   #changement type  ==> string
df_name_basics['knownForTitles'] = df_name_basics['knownForTitles'].astype("string")         #changement type  ==> string




In [19]:
# RENOMME LA COLONNE EN TCONST POUR MERGE PAR LA SUITE AVEC LE MEME NOM DE COLONNE QUE LES AUTRES TABLES.   

df_name_basics = df_name_basics.rename(columns = {'knownForTitles' : 'tconst'})    

In [21]:
# GET DUMMIES POUR AVOIR LES PROFESSIONS EN COLONNE BOOELEEN POUR LE MACHINE LEARNING

df_act_dummies = df_name_basics['primaryProfession'].str.get_dummies(sep=',')

df_name_basics = pd.concat([df_name_basics, df_act_dummies], axis = 1)

In [22]:
# EXPLODE DES ID DES FILMS ET ON GARDE DANS LE DF LES COLONNES QUE L'ON SOUHAITE GARDER POUR LA SUITE.

df_name_basics['tconst'] = df_name_basics['tconst'].str.split(",")
df_name_basics = df_name_basics.explode('tconst')


df_name_basics = df_name_basics[['nconst', 'primaryName', 'birthYear', 'deathYear', 'tconst', 'actor', 'actress', 'director', 'producer', 'writer']]

In [26]:
# RESULTAT :

df_name_basics.head(5)

,nconst,primaryName,birthYear,deathYear,tconst,actor,actress,director,producer,writer
0,nm0000001,Fred Astaire,1899,1987,tt0072308,1,0,0,1,0
0,nm0000001,Fred Astaire,1899,1987,tt0050419,1,0,0,1,0
0,nm0000001,Fred Astaire,1899,1987,tt0053137,1,0,0,1,0
0,nm0000001,Fred Astaire,1899,1987,tt0027125,1,0,0,1,0
1,nm0000002,Lauren Bacall,1924,2014,tt0037382,0,1,0,0,0


**TABLE TITLE BASIC :**

In [27]:
df_title_basics = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\title.basics.tsv', sep ='\t')

C:\Users\33660\AppData\Local\Temp\ipykernel_22156\2333409084.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\title.basics.tsv', sep ='\t')


In [28]:
# SUPPRESSION DES VALEURS NULL QUANT ELLES NE SONT PAS PERTINANTES.
# SUPPRESSION DES FILMS POUR ADULTES
# FILTRE SUR LES FILM AVANT 1950 ET SUR LES TYPES DE FILMS POUR NE GARDER QUE LES FILMS ET SERIES.
# SUPPRESSION DES COLONNES INUTILES.
# CHANGEMENT DE TYPES DES COLONNES
# SUPPRESSION ET FILTRE DES FILMS AYANT DES VALEURS EXTRÊMES EN TERMES DE DUREE.


df_title_basics = df_title_basics.replace(r"\N", pd.NA)  
df_title_basics = df_title_basics.dropna(subset = ['startYear'])

df_title_basics = df_title_basics.drop(df_title_basics[df_title_basics['isAdult'] == 1].index)

df_title_basics = df_title_basics.drop(df_title_basics[df_title_basics['startYear'] < '1950'].index)

df_title_basics = df_title_basics.dropna(subset = ['genres'])

df_title_basics = df_title_basics.drop(df_title_basics[(df_title_basics['titleType'] != 'movie') & (df_title_basics['titleType'] != 'tvSeries')].index)

df_title_basics = df_title_basics.drop(['primaryTitle', 'isAdult'],axis=1)


df_title_basics = df_title_basics.dropna(subset = ['runtimeMinutes'])


df_title_basics = df_title_basics.astype({ 
    'tconst': str,
    'titleType': str,
    'originalTitle': str,
    'genres': str,
    'startYear': str,
    'endYear': str,
    'runtimeMinutes' : int })

df_title_basics = df_title_basics.drop(df_title_basics[df_title_basics['runtimeMinutes'] < 60].index)

df_title_basics = df_title_basics.drop(df_title_basics[df_title_basics['runtimeMinutes'] > 240].index)

df_title_basics.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres
13252,tt0013274,movie,Istoriya grazhdanskoy voyny,2021,<NA>,94,Documentary
15699,tt0015724,movie,Dama de noche,1993,<NA>,102,"Drama,Mystery,Romance"
22019,tt0022064,movie,Lebbra bianca,1951,<NA>,80,Drama
25504,tt0025557,movie,El negro que tenía el alma blanca,1951,<NA>,87,"Drama,Musical"
30934,tt0030998,movie,You're Only Young Twice,1952,<NA>,81,Comedy


**TITLE RATINGS :**

In [29]:
df_title_ratings = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\title.ratings.tsv', sep ='\t')

In [30]:
#check results
count_info = df_title_ratings.count()
print(count_info)

#calcul de la noté pondérée
C = df_title_ratings['averageRating'].mean()
m = df_title_ratings['numVotes'].quantile(0.9)


#tri
df_title_ratings = df_title_ratings[df_title_ratings['numVotes'] >= m].sort_values('weightedRating', ascending=False)

#type to str
df_title_ratings['numVotes'] = df_title_ratings['numVotes'].astype(str)
df_title_ratings['averageRating'] = df_title_ratings['averageRating'].astype(str)
df_title_ratings['weightedRating'] = df_title_ratings['weightedRating'].astype(str)


df_title_ratings.head()

tconst           1421253
averageRating    1421253
numVotes         1421253
dtype: int64


,tconst,averageRating,numVotes,weightedRating
929099,tt2301451,10.0,213953,9.99
900781,tt2178784,9.9,117173,9.89
1134873,tt4283094,9.9,159336,9.89
1134871,tt4283088,9.9,223499,9.89
929101,tt2301455,9.9,138596,9.89


**TABLE IMDB** :

In [ ]:
df_option = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\tmdb_full.csv')

In [ ]:
# SUPPRESSION DES COLONNES INUTILES
# SUPPRESSION DES VALEURS NULL SUR LES COLONNES PERTINENTE
# SUPPRESSION DES FILM AVANT 1950
# FILTRE SUR LES STATUS DES FILMS.


df_option = df_option.drop(['adult', 'homepage', 'video', 'production_companies_country', 'tagline', 'original_title', 'id'], axis = 1)  

df_option = df_option.rename(columns={'imdb_id' : 'tconst'})

df_option = df_option.dropna(subset=['poster_path'])
df_option = df_option.dropna(subset=['release_date'])
df_option = df_option.drop(df_option[df_option['release_date'] < '1950-01-01'].index) 

df_option['release_date'] = pd.to_datetime(df_option['release_date'])

df_option = df_option[(df_option['status'] == 'Released') | (df_option['status'] == 'In Production') | (df_option['status'] == 'Post Production') | (df_option['status'] == 'Planned')]


In [ ]:
df_option.to_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\df_fichier_option_imdb.csv', sep =',')

**JOINTURE DE TOUTE LES TABLES AFIN D'EN AVOIR UNE SEULE POUR LE MACHINE LEARNING** :


In [ ]:
df_imdb = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\df_fichier_option_imdb.csv', sep =',')

In [71]:
# JOINTURE ENTRE LES TABLES

df_big_table = pd.merge(df_title_basics, df_title_ratings, on = 'tconst', how = 'left')

df_big_table = pd.merge(df_big_table, df_imdb, on = 'tconst', how = 'outer')

1er Nettoyage de la partie big table :

In [72]:

# uniformisation des lignes de genres_y pour la concatener avec genres_x et pouvoir faire le str.get_dummies(sep = ",")
df_big_table['genres_y'] = df_big_table['genres_y'].str.replace("[", "").str.replace("'", "").str.replace("]", "")

# Suppression valeur NULL pour les affiches de film :
df_big_table = df_big_table.dropna(subset = ['poster_path'])

# Fonction combine first() --> permet de fusionner 2 colonnes en remplaçant les valeurs NULL de la 1ère par les valeurs non NULL de la deuxième colonne:
df_big_table['genres'] = df_big_table['genres_x'].combine_first(df_big_table['genres_y'])                
df_big_table['durée_du_film'] = df_big_table['runtimeMinutes'].combine_first(df_big_table['runtime']) 
df_big_table['moyenne_vote'] = df_big_table['averageRating'].combine_first(df_big_table['vote_average'])
df_big_table['titre_du_film'] = df_big_table['originalTitle'].combine_first(df_big_table['title']) 

# suppression des colonnes devenu inutile :
colonnes_suppr = ['startYear', 'weightedRating', 'endYear', 'numVotes', 'averageRating', 'vote_average', 'runtimeMinutes', 'runtime', 'genres_x', 'genres_y', 'title', 'originalTitle', 'backdrop_path', 'titleType', 'status', 'production_countries', 'spoken_languages']
df_big_table = df_big_table.drop(colonnes_suppr, axis = 1)  

# valeur NA dans les valeurs manquante de genres

df_big_table['genres'] = df_big_table['genres'].replace('', pd.NA)

# puis suppression des valeurs nul dans genres
df_big_table = df_big_table.dropna(subset=['genres'])


GET DUMMIES IMPORTANT SUR LA COLONNE GENRES QUI VA DETERMINER LA BASE DE NOTRE ML POUR RESSORTIR UN FILM EN PRIORITE PAR GENRES DE FILM.

In [74]:
# DF avec un get_dummies sur les genres en precisant à partir de qu'elle caractère séparer les genres (str.get_summies(sep=','))

df_dummy = df_big_table['genres'].str.get_dummies(sep=',')

In [75]:
df_dummy.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Science Fiction,Sport,TV Movie,Talk-Show,Thriller,War,Western
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [76]:
# Concatenation avec la big table

df_big_table = pd.concat([df_big_table, df_dummy], axis = 1)

# suppression de la colonne genre
df_big_table = df_big_table.drop('genres', axis = 1)

# colonne release_date en datetime
df_big_table['release_date'] = pd.to_datetime(df_big_table['release_date'])

# colonne moyenne_vote en float
df_big_table['moyenne_vote'] = df_big_table['moyenne_vote'].astype(float)

JOINTURE AVEC DF_NAME_BASIC POUR AVOIR LES ACTEURS ET ACTRICES DE CHAQUES FILMS:


In [82]:
df_big_table = pd.merge(df_big_table, df_name_basics, on = 'tconst', how = 'inner')

In [95]:
df_big_table.head(2)

,tconst,budget,original_language,overview,popularity,poster_path,release_date,revenue,vote_count,production_companies_name,durée_du_film,moyenne_vote,titre_du_film,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Science Fiction,Sport,TV Movie,Talk-Show,Thriller,War,Western,nconst,primaryName,birthYear,deathYear,actor,actress,director,producer,writer
0,tt0015724,0.0,en,"Bruno, a novelist with no luck goes to call fo...",0.833,/zCmyAl7VG6aZJqWZ7PFfj9e6ToU.jpg,1993-05-18,0.0,6.0,['Centro de Capacitación Cinematográfica (CCC)'],102.0,7.0,Dama de noche,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,nm0529960,Eva López Sánchez,1954,<NA>,0,0,1,0,1
1,tt0015724,0.0,en,"Bruno, a novelist with no luck goes to call fo...",0.833,/zCmyAl7VG6aZJqWZ7PFfj9e6ToU.jpg,1993-05-18,0.0,6.0,['Centro de Capacitación Cinematográfica (CCC)'],102.0,7.0,Dama de noche,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,nm0600039,Gustavo Montiel Pagés,1954,2012,1,0,1,1,0


NORMALISATION DES DONNEES POUR ENTRAINER LE MACHINE LEARNING :

In [84]:
# on créer une table contenant uniquement les données numériques qui vont être normalisées.
df_num = df_big_table.select_dtypes(include='number').drop('birthYear', axis = 1)

# on créer une table avec le reste des colonnes qui ne seront pas normalisées.
df_classes = df_big_table.drop(df_num, axis = 1)

In [86]:
# On entraine et on transforme les données pour la normalisation :

from sklearn.preprocessing import MinMaxScaler

scaler_minmax = MinMaxScaler()

df_num = pd.DataFrame(scaler_minmax.fit_transform(df_num), columns = df_num.columns)

In [87]:
# on concat ensuite les df ensembles une fois transformés.

df_ML = pd.concat([df_classes, df_num], axis = 1)

In [88]:
df_ML.head(2)

,tconst,original_language,overview,poster_path,release_date,production_companies_name,titre_du_film,nconst,primaryName,birthYear,deathYear,budget,popularity,revenue,vote_count,durée_du_film,moyenne_vote,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Science Fiction,Sport,TV Movie,Talk-Show,Thriller,War,Western,actor,actress,director,producer,writer
0,tt0015724,en,"Bruno, a novelist with no luck goes to call fo...",/zCmyAl7VG6aZJqWZ7PFfj9e6ToU.jpg,1993-05-18,['Centro de Capacitación Cinematográfica (CCC)'],Dama de noche,nm0529960,Eva López Sánchez,1954,<NA>,0.0,0.000046,0.0,0.000178,0.002361,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,tt0015724,en,"Bruno, a novelist with no luck goes to call fo...",/zCmyAl7VG6aZJqWZ7PFfj9e6ToU.jpg,1993-05-18,['Centro de Capacitación Cinematográfica (CCC)'],Dama de noche,nm0600039,Gustavo Montiel Pagés,1954,2012,0.0,0.000046,0.0,0.000178,0.002361,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [96]:
df_ML.to_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\df_ml.csv', index = False)

---

---

**PARTIE MACHINE LEARNING : ENTRAINEMENT D'UN MODELE KNN** :

In [74]:
import warnings
warnings.filterwarnings('ignore', category = UserWarning)

**MODELE KNN MACHINE LEARNING** :

In [93]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np


# Fonction de recommandation avec prise en compte des poids
def recommandation(film):
    
    df_ML = pd.read_csv('C:\\Users\\33660\\Desktop\\PROJET 2\\df_ml.csv')

    df_filtre = df_ML.groupby('titre_du_film').first().reset_index()
    
    # Filtrer les colonnes numériques pour le modèle KNN
    X = df_filtre.select_dtypes(include='number').drop(['birthYear', 'deathYear'], axis=1, errors='ignore')
    
    # Supprimer les colonnes qui ne seront pas pondérées
    weights = X.drop(['budget', 'popularity', 'revenue', 'vote_count', 'durée_du_film', 'moyenne_vote'], axis=1)
    
    # Multiplier les poids par 2
    weights *= 2
    
    # Concaténer les colonnes pondérées avec les colonnes qui ne seront pas pondérées
    X_weighted = pd.concat([X[['budget', 'popularity', 'revenue', 'vote_count', 'durée_du_film', 'moyenne_vote']], weights], axis=1)
    
    # Entraîner le modèle KNN avec les données pondérées
    distanceKNN = NearestNeighbors(n_neighbors=3).fit(X_weighted)
    
    # Extraire les caractéristiques du film à recommander
    user_df = df_filtre.loc[df_filtre['titre_du_film'] == film, X.columns]
    
    # Trouver les voisins les plus proches
    distance, indices = distanceKNN.kneighbors(user_df)
    
    # Afficher les films recommandés
    for idx in indices[0][1]:  # Utiliser indices[0][1] pour accéder aux indices des voisins en excluant lui même
        print(df_filtre.iloc[idx]['titre_du_film'])

# Appeler la fonction de recommandation pour 'Avatar'
recommandation('Avatar')



Avatar
The Hobbit: An Unexpected Journey
Doctor Strange


---

---